# Анализ сервиса вопросов и ответов по программированию


### Описание проекта


- В самостоятельном проекте вы будете работать с базой данных StackOverflow — сервиса вопросов и ответов о программировании. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.


- Вы будете работать с версией базы, где хранятся данные о постах за 2008 год, но в таблицах вы найдёте информацию и о более поздних оценках, которые эти посты получили. 


### Описание данных

**Таблица stackoverflow.badges**

Хранит информацию о значках, которые присуждаются за разные достижения. Например, пользователь, правильно ответивший на большое количество вопросов про PostgreSQL, может получить значок postgresql. 

- id	Идентификатор значка, первичный ключ таблицы


- name	Название значка


- user_id	Идентификатор пользователя, которому присвоили значок, внешний ключ, отсылающий к таблице users


- creation_date	Дата присвоения значка



**Таблица stackoverflow.post_types**

Содержит информацию о типе постов. Их может быть два:

           - Question — пост с вопросом;
           - Answer — пост с ответом.

- id	Идентификатор поста, первичный ключ таблицы


- type	Тип поста


**Таблица stackoverflow.posts**

Содержит информацию о постах.


- id	Идентификатор поста, первичный ключ таблицы


- title	Заголовок поста


- creation_date	Дата создания поста


- favorites_count	Число, которое показывает, сколько раз пост добавили в «Закладки»


- last_activity_date	Дата последнего действия в посте, например комментария


- last_edit_date	Дата последнего изменения поста


- user_id	Идентификатор пользователя, который создал пост, внешний ключ к таблице users


- parent_id	Если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом


- post_type_id	Идентификатор типа поста, внешний ключ к таблице post_types


- score	Количество очков, которое набрал пост


- views_count	Количество просмотров


**Таблица stackoverflow.users**

Содержит информацию о пользователях.

- id	Идентификатор пользователя, первичный ключ таблицы


- creation_date	Дата регистрации пользователя


- display_name	Имя пользователя


- last_access_date	Дата последнего входа


- location	Местоположение


- reputation	Очки репутации, которые получают за хорошие вопросы и полезные ответы


- views	Число просмотров профиля пользователя


**Таблица stackoverflow.vote_types**

Содержит информацию о типах голосов. Голос — это метка, которую пользователи ставят посту. Типов бывает несколько: 

          - UpMod — такую отметку получают посты с вопросами или ответами, которые пользователи посчитали уместными и полезными.
          - DownMod — такую отметку получают посты, которые показались пользователям наименее полезными.
          Close — такую метку ставят опытные пользователи сервиса, если заданный вопрос нужно доработать или он вообще не подходит для платформы.
          - Offensive — такую метку могут поставить, если пользователь ответил на вопрос в грубой и оскорбительной манере, например, указав на неопытность автора поста.
          - Spam — такую метку ставят в случае, если пост пользователя выглядит откровенной рекламой.

- id	Идентификатор типа голоса, первичный ключ


- name	Название метки


**Таблица stackoverflow.votes**

Содержит информацию о голосах за посты. 


- id	Идентификатор голоса, первичный ключ


- post_id	Идентификатор поста, внешний ключ к таблице posts


- user_id	Идентификатор пользователя, который поставил посту голос, внешний ключ к таблице users


- bounty_amount	Сумма вознаграждения, которое назначают, чтобы привлечь внимание к посту


- vote_type_id	Идентификатор типа голоса, внешний ключ к таблице vote_types


- creation_date	Дата назначения голоса


### [ER-диаграмма](https://disk.yandex.ru/i/lbSQWWbjUBI5fw)

### Задача 1

**Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».**

SELECT COUNT(pt.type)

FROM stackoverflow.post_types AS pt JOIN stackoverflow.posts AS p ON p.post_type_id = pt.id

WHERE (p.score > 300 OR p.favorites_count >= 100)

AND pt.type = 'Question'

### Задача 2

**Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.**

SELECT *,

      DENSE_RANK() OVER (ORDER BY b.b_cnt DESC) AS rating
      
FROM (SELECT user_id,

             COUNT(id) AS b_cnt
             
      FROM stackoverflow.badges
      
      WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15' 
      
      GROUP BY user_id
      
      ORDER BY b_cnt DESC, user_id LIMIT 10) as b;

### Задача 3

**Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.**

SELECT COUNT(DISTINCT b.user_id)

FROM stackoverflow.badges AS b JOIN stackoverflow.users AS u ON b.user_id=u.id

WHERE u.creation_date::date = b.creation_date::date

### Задача 4

**Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?**

SELECT COUNT(DISTINCT(cv.id))

FROM (SELECT ps.id

      FROM stackoverflow.posts AS ps
      
      JOIN stackoverflow.votes AS v ON ps.id=v.post_id
      
      JOIN stackoverflow.users AS u ON ps.user_id=u.id
      
      WHERE u.display_name LIKE 'Joel Coehoorn' AND v.id > 0
      
      GROUP BY ps.id) AS cv;

### Задача 5

**Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.**

SELECT *,

       ROW_NUMBER() OVER (ORDER BY id DESC)
       
FROM stackoverflow.vote_types

ORDER BY id; 

### Задача 6

**Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.**

SELECT DISTINCT users,

                count_votes
                
FROM

(SELECT u.id users,

       COUNT(vt.id) count_votes
       
FROM stackoverflow.users AS u 

JOIN stackoverflow.votes AS v ON u.id=v.user_id

JOIN stackoverflow.vote_types AS vt ON vt.id=v.vote_type_id

WHERE vt.name = 'Close'

GROUP BY u.id) AS kk

ORDER BY count_votes DESC, users DESC

LIMIT 10

### Задача 7

**Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.

- **Отобразите несколько полей:**

- **идентификатор пользователя;**

- **число значков;**

- **место в рейтинге — чем больше значков, тем выше рейтинг.**

**Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.**

**Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.**

SELECT user_id, count_badges, DENSE_RANK() OVER (ORDER BY count_badges DESC) rating

FROM (

    SELECT u.id user_id, COUNT(DISTINCT b.id) count_badges
    
    FROM stackoverflow.users AS u
    
    JOIN stackoverflow.badges AS b ON u.id = b.user_id
    
    WHERE b.creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
    
    GROUP BY u.id
    
) AS badge_counts

ORDER BY count_badges DESC, user_id

LIMIT 10;

### Задача 8

**Сколько в среднем очков получает пост каждого пользователя?

Сформируйте таблицу из следующих полей:

- **заголовок поста;**

- **идентификатор пользователя;**

- **число очков поста;**

- **среднее число очков пользователя за пост, округлённое до целого числа.**

**Не учитывайте посты без заголовка, а также те, что набрали ноль очков.**

SELECT title_post,

       user_id,
       
       score_post,
       
       ROUND(AVG(score_post) OVER (PARTITION BY user_id))
       
FROM

(SELECT p.title AS title_post,

        u.id AS user_id,
        
        p.score AS score_post
        
 FROM stackoverflow.posts AS p 
 
 JOIN stackoverflow.users AS u ON p.user_id = u.id
 
 WHERE p.title IS NOT NULL AND p.score <> 0
 
 GROUP BY p.title, u.id, p.score) AS kk;

### Задача 9

**Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.**

SELECT title

FROM stackoverflow.posts

WHERE user_id IN (SELECT user_id

                  FROM stackoverflow.badges
                  
                  GROUP BY user_id
                  
                  HAVING COUNT(id) >1000)
                  
              AND title IS NOT NULL;

### Задача 10

**Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:**

- **пользователям с числом просмотров больше либо равным 350 присвойте группу 1;**

- **пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;**

- **пользователям с числом просмотров меньше 100 — группу 3.**

**Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу.**

**Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.**

SELECT id, 

       views,
       
       CASE
       
       WHEN views >= 350 THEN 1
       
       WHEN views < 350 AND views >= 100 THEN 2
       
       WHEN views < 100 THEN 3
       
       END
       
FROM stackoverflow.users 

WHERE location LIKE '%United States%'

AND views > 0

### Задача 11

**Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.**

WITH grp AS (SELECT g.id,

                    g.views,
                    
                    g.group,
                    
                    MAX(g.views) OVER (PARTITION BY g.group) AS max    
                    
             FROM (SELECT id,
             
                          views,
                          
                          CASE
                          
                             WHEN views >= 350 THEN 1
                             
                             WHEN views < 100 THEN 3
                             
                             ELSE 2
                             
                          END AS group
                          
                   FROM stackoverflow.users
                   
                   WHERE location LIKE '%United States%' AND views > 0) as g
                   
              )
              
  
SELECT grp.id, 

       grp.views,  
       
       grp.group
       
       
FROM grp

WHERE grp.views = grp.max

ORDER BY grp.views DESC, grp.id;

### Задача 12

**Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:**

- **номер дня;**

- **число пользователей, зарегистрированных в этот день;**

- **сумму пользователей с накоплением.**

SELECT 

  EXTRACT(DAY FROM creation_date) AS day_number, 
  
  COUNT(*) AS new_users_count, 
  
  SUM(COUNT(*)) OVER (ORDER BY EXTRACT(DAY FROM creation_date)) AS cumulative_users_count
  
FROM 

  stackoverflow.users
  
WHERE 

  EXTRACT(YEAR FROM creation_date) = 2008 AND EXTRACT(MONTH FROM creation_date) = 11
  
GROUP BY 

  EXTRACT(DAY FROM creation_date)
  
ORDER BY 

  EXTRACT(DAY FROM creation_date)

### Задача 13

**Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:**

- **идентификатор пользователя;**

- **разницу во времени между регистрацией и первым постом.**

WITH dt AS (SELECT DISTINCT user_id,

                            MIN(creation_date) OVER (PARTITION BY user_id) AS min_dt      
                            
            FROM stackoverflow.posts)

SELECT dt.user_id,

       (dt.min_dt - u.creation_date) AS diff
       
FROM stackoverflow.users AS u 

JOIN dt ON  u.id = dt.user_id;

### Задача 14

**Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.**

SELECT DATE_TRUNC('month', creation_date)::date,

       SUM(views_count) AS sum_views 
       
FROM stackoverflow.posts

WHERE EXTRACT(YEAR FROM creation_date) = 2008

GROUP BY DATE_TRUNC('month', creation_date)

ORDER BY sum_views DESC;

### Задача 15

**Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.**

SELECT u.display_name,

       COUNT(DISTINCT(p.user_id))
       
FROM stackoverflow.users AS u JOIN stackoverflow.posts AS p ON u.id=p.user_id

JOIN stackoverflow.post_types AS pt ON p.post_type_id=pt.id

WHERE p.creation_date::date BETWEEN u.creation_date::date AND u.creation_date::date + INTERVAL '1 month'

      AND pt.type = 'Answer'
      
GROUP BY u.display_name

HAVING COUNT(p.id) > 100

ORDER BY u.display_name;

### Задача 16

**Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.**

WITH users AS (SELECT u.id

               FROM stackoverflow.posts AS p
               
               JOIN stackoverflow.users AS u ON p.user_id=u.id
               
               WHERE DATE_TRUNC('month', u.creation_date)::date = '2008-09-01' 
               
                   AND DATE_TRUNC('month', p.creation_date)::date = '2008-12-01'
                   
               GROUP BY u.id
               
               HAVING COUNT(p.id) > 0)
               

SELECT COUNT(p.id),

       DATE_TRUNC('month', p.creation_date)::date
       
FROM stackoverflow.posts AS p

WHERE p.user_id IN (SELECT *

                    FROM users)
                    
      AND DATE_TRUNC('year', p.creation_date)::date = '2008-01-01'
      
GROUP BY DATE_TRUNC('month', p.creation_date)::date

ORDER BY DATE_TRUNC('month', p.creation_date)::date DESC;

### Задача 17

**Используя данные о постах, выведите несколько полей:**

- **идентификатор пользователя, который написал пост;**

- **дата создания поста;**

- **количество просмотров у текущего поста;**

- **сумму просмотров постов автора с накоплением.**

**Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.**

SELECT user_id,

       creation_date,
       
       views_count,
       
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
       
FROM stackoverflow.posts;

### Задача 18

**Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.**

WITH ad AS (SELECT user_id,

                   COUNT(DISTINCT creation_date::date)  AS cnt
                   
            FROM stackoverflow.posts
            
            WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07' 
            
            GROUP BY user_id)
            

SELECT ROUND(AVG(ad.cnt))

FROM ad;

### Задача 19

**На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:**

- **номер месяца;**

- **количество постов за месяц;**

**процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим. Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.**

WITH month_post AS (SELECT EXTRACT(MONTH from creation_date::date) AS month,

                    COUNT(DISTINCT id) count
                    
                    FROM stackoverflow.posts
                    
                    WHERE creation_date::date BETWEEN '2008-09-01' AND '2008-12-31'
                    
                    GROUP BY month)
                    

SELECT *,

       ROUND(((count::numeric / LAG(count) OVER (ORDER BY month)) - 1) * 100, 2) AS user_growth
       
FROM month_post;

### Задача 20

**Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:**

- **номер недели;**

- **дата и время последнего поста, опубликованного на этой неделе.**

WITH user_post AS (SELECT user_id,

                   COUNT(DISTINCT id) AS cnt
                   
                   FROM stackoverflow.posts
                   
                   GROUP BY user_id
                   
                   ORDER BY cnt DESC
                   
                   LIMIT 1),
                   

     dtt AS (SELECT p.user_id,
     
             p.creation_date,
             
             extract('week' from p.creation_date) AS week_number
             
             FROM stackoverflow.posts AS p
             
             JOIN user_post ON user_post.user_id = p.user_id
             
             WHERE DATE_TRUNC('month', p.creation_date)::date = '2008-10-01')
             

SELECT DISTINCT week_number::numeric,

       MAX(creation_date) OVER (PARTITION BY week_number) AS post_dt
       
FROM dtt

ORDER BY week_number;